# Bedrock Claude API
- Application/end users -> App server - Bedrock Client -> AWS Bedrock (with User message & Model ID)
- Focus of course - Server/Bedrock Client -> AWS Bedrock
- Bedrock API vs Anthropic API - both hosting same LLMs - but different API and SDKs
- Request to Bedrock - Region, Model ID, User Message
- Not every model available in every Region
- Inference Profiles - route requests to region where model hosted (higher throughput & performance)  - use in Model ID. Also needed for monitoring usage (CW Metrics) and Cost Tracking (Application inference profiles; user-defined). System-defined vs User-defined Inference Profiles. Also - Global Inference profiles (target all commercial regions)


## Bedrock Runtime


In [ ]:
import boto3
from dotenv import load_dotenv
import os

load_dotenv()

client = boto3.client("bedrock-runtime", region_name="us-east-1",
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
                      )

inference_profile_id = 'us.anthropic.claude-3-5-haiku-20241022-v1:0' #Global inference profile
model_id = inference_profile_id

In [ ]:
# #Multi-Turn conversation
#
# # Make a starting list of messages
# messages = []
#
# # Add in the initial user question of "What's 1+1?"
# add_user_message(messages, "What's 1+1?")
#
# # Pass the list of messages into chat to get an answer
# answer = chat(messages)
#
# # Take the answer and add it as an assistant message into our list
# add_assistant_message(messages, answer)
#
# # Add in the user's followup question
# add_user_message(messages, "And 3 more added to that?")
#
# # Call chat again with the list of messages to get a final answer
# answer = chat(messages)
# print(answer)

In [ ]:
# # Chatbot
# messages = []
#
# while True:
#     user_input = input(">")
#     add_user_message(messages, user_input)
#     answer=chat(messages)
#     add_assistant_message(messages, answer)
#     print(answer)
#


# Multi-Turn conversations (CONVERSE)
- Converse Function
- Configuration parameters (inferenceConfig) - maxTokens, temperature, StopSequences...etc
- Prefilled Assistant Messages
- System Prompts
- Structured Data enforcing (for model output) = Pre-filled assistant message + StopSequences: CSV, JSON...any (you might need to use text.strip() or library like csv/JSON etc to manipulate output, clean up (e.g. newline, whitespaces...etc)

In [60]:
#Helper Functions
def add_user_message(messages, text):
    user_message = {
        "role": "user",
        "content": [
            {"text": text}
        ]
    }
    messages.append(user_message)

def add_assistant_message(messages, text):
    assistant_message = {
        "role": "assistant",
        "content": [
            {"text": text}
        ]
    }
    messages.append(assistant_message)

def chat(messages, temperature=1.0, system=None, stop_sequences=[]):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "maxTokens": 1000,
            "stopSequences": stop_sequences
        },
    }

    if system:
        params["system"] = [{"text": system}]

    response = client.converse(**params)

    return response["output"]["message"]["content"][0]["text"]

In [ ]:
#Standard conversation
# messages = []
#
# add_user_message(messages, "How di I host a PostgreSQL database?")
# text = chat(messages, system="You are an AWS Support specialist and need to answer user queries.")

In [56]:
# #Pre-filling Assistant message (i.e. biasing response)
# messages = []
#
# add_user_message(messages, "Is coffee or  tea better for breakfast?")
# add_assistant_message(messages, "They are the same because")
#
# chat(messages)

' everyone has different preferences. Some key differences:\n\nCoffee:\n- Higher caffeine content\n- May boost metabolism\n- Can aid weight loss\n- Often associated with more energy\n- More bitter taste\n\nTea:\n- Lower caffeine content\n- Contains antioxidants\n- Often gentler on stomach\n- Variety of flavors\n- Can have health benefits like reducing inflammation\n\nThe "better" choice depends on your personal taste, health goals, and how your body responds to each beverage.'

In [65]:
#Structured Data
#Using Pre-filling message + Stop sequences

messages = []

prompt = """
Generate three different sample AWS CLI commands. Each should be very short.
"""

add_user_message(messages, prompt)
add_assistant_message(
    messages,
    "Here are all three commands in a single block without any comments:\n```bash",
)

text = chat(messages, stop_sequences=["```"])
text.strip()


'aws s3 ls\naws ec2 describe-instances\naws iam list-users'

In [66]:
#Cleaning up structured data output
import json

# Parse as JSON to validate and format
parsed_data = json.loads(text.strip())

#Other data types text.strip()
parsed_data


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Streaming Conversations
- ```converse_stream()```
- ContextBlockDelta events key

In [54]:
messages = []
add_user_message(messages, "How di I host a PostgreSQL database?")
response = client.converse_stream(messages=messages, modelId=model_id)
for event in response["stream"]:
    if "contentBlockDelta" in event:
        chunk = event["contentBlockDelta"]["delta"]["text"]
        print(chunk, end="")
        text += chunk

print("\n\nTotal Message:\n" + text)


Here are several ways to host a PostgreSQL database:

1. Local Installation
- Download PostgreSQL from official website
- Install on your local machine
- Use default localhost connection

2. Cloud Platforms
a) Amazon RDS
- Managed PostgreSQL service
- Easy setup
- Scalable and secure
- Pay per usage

b) Google Cloud SQL
- Fully managed PostgreSQL database
- Automatic backups
- Automatic replication

c) Azure Database for PostgreSQL
- Managed service by Microsoft
- Supports single server and flexible server modes
- Built-in security features

3. Self-Hosted Options
a) Virtual Private Server (VPS)
- Providers like DigitalOcean, Linode
- Full control over database
- Manual configuration required

b) Kubernetes
- Container-based deployment
- Using Helm charts
- Scalable and portable

4. Managed Hosting Services
- Heroku Postgres
- ElephantSQL
- Render
- Supabase

5. Docker Deployment
```bash
docker run --name postgres-db \
  -e POSTGRES_PASSWORD=mypassword \
  -p 5432:5432 \
  -d postgres
